Scrapping using beautifulsoup

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
import requests
from bs4 import BeautifulSoup
import csv

# Kata kunci pencarian
kata_kunci = input("Masukkan kata kunci: ")
kata_kunci = kata_kunci.replace(' ', '+')

sort_input = int(input('''pilih urutan berdasarkan
ketik 1 untuk artikel yang relevan
ketik 0 untuk artikel yang terbaru :
'''))

# Atur parameter urutan berdasarkan input pengguna
sort = 'Relevance' if sort_input == 1 else 'PubDate'

# Jumlah baris maksimum
max_rows = int(input("Masukkan jumlah artikel yang dicari: "))

# Membuka file CSV
with open('/content/drive/MyDrive/pemrosesan text/hasil_scraping.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    # Menulis header
    writer.writerow(["No", "Judul Artikel", "Nama Jurnal", "Prosiding", "Penulis", "Tahun Terbit", "URL Artikel", "URL PDF", "Abstrak"])

    # Inisialisasi nomor baris
    row_number = 0

    # Looping halaman
    for page in range(1, max_rows):
        # Jika jumlah baris maksimum telah tercapai, hentikan loop
        if row_number >= max_rows:
            break

        # URL SpringerOpen
        url = f"https://www.springeropen.com/search?searchType=publisherSearch&sort={sort}&query={kata_kunci}&page={page}"

        # Membuat permintaan ke SpringerOpen
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')

        # Mencari semua elemen dengan class 'c-list-group__item'
        results = soup.find_all('li', {'class': 'c-listing__item u-keyline'})

        # Looping melalui setiap hasil dan mengekstrak informasi
        for result in results:
            # Tambahkan nomor baris
            row_number += 1

            # Jika jumlah baris maksimum telah tercapai, hentikan loop
            if row_number > max_rows:
                break

            title = result.find('h3', {'class': 'c-listing__title'}).text if result.find('h3', {'class': 'c-listing__title'}) else "N/A"
            journal_info = result.find('em',{'data-test': 'journal-title'}).text if result.find('em',{'data-test': 'journal-title'}) else "N/A"
            url_article_element = result.find('a')
            url_article = "https:" + url_article_element['href'] if url_article_element and url_article_element.has_attr('href') else "N/A"

            author = result.find('span', {'class': 'c-listing__authors-list'}).text if result.find('span', {'class': 'c-listing__authors-list'}) else "N/A"
            year_published = result.find('span', {'itemprop': 'datePublished'}).text if result.find('span', {'itemprop': 'datePublished'}) else "N/A"

            pdf_link_element = result.find('a', {'data-test': 'pdf-link'})
            pdf_link = "https:" + pdf_link_element['href'] if pdf_link_element and pdf_link_element.has_attr('href') else "N/A"

            # Membuat permintaan ke halaman artikel
            url_abstract_element = result.find('a', itemprop='url')
            abstract_link = "https:" + url_abstract_element['href']
            article_response = requests.get(abstract_link)
            article_soup = BeautifulSoup(article_response.text, 'html.parser')

            # Temukan div dengan class 'c-article-section__content'
            result_abstract = article_soup.find('div', {'class': 'c-article-section__content'})

            # Ekstrak teks dalam tag <p> di dalam div
            abstract = result_abstract.find('p').text if result_abstract and result_abstract.find('p') else "N/A"

            # Mengubah abstrak menjadi teks biasa (ASCII)
            abstract_plain = abstract.encode('ascii', 'ignore').decode('ascii')

            # Menghapus spasi ekstra
            abstract_plain = " ".join(abstract_plain.split())

            # Menulis baris ke file CSV
            writer.writerow([row_number, title, journal_info, "N/A", author, year_published, url_article, pdf_link, abstract_plain])


Masukkan kata kunci: speech to text 
pilih urutan berdasarkan
ketik 1 untuk artikel yang relevan
ketik 0 untuk artikel yang terbaru :
1
Masukkan jumlah artikel yang dicari: 10
